imports and stuff


In [4]:
import sys
import numpy as np
import pandas as pd
sys.path.append('../')
full_data = pd.read_csv("../datasets/agoda_cancellation_train.csv")
full_data.head()


,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
0,-9223194055642672935,2018-06-28 21:15:00,2018-07-09 00:00:00,2018-07-13 00:00:00,6452,HK,2009-06-28 02:02:00,4.0,Hotel,Pay Now,...,NaN,NaN,NaN,NaN,NaN,NaN,1192,NaN,NaN,142
1,-9222713784330706132,2018-08-10 22:31:00,2018-08-16 00:00:00,2018-08-17 00:00:00,47729,CN,2011-06-07 11:52:00,4.0,Hotel,Pay Later,...,0.0,0.0,1.0,0.0,0.0,NaN,5617,902.0,600.0,1273
2,-9222411208325704942,2018-09-14 07:55:00,2018-09-14 00:00:00,2018-09-15 00:00:00,780431,KR,2014-11-20 15:43:00,4.0,Hotel,Pay Now,...,0.0,0.0,0.0,0.0,0.0,NaN,3819,NaN,NaN,2799
3,-9222220845872895471,2018-06-25 07:33:00,2018-07-02 00:00:00,2018-07-03 00:00:00,291365,JP,2011-11-21 12:27:00,3.0,Hotel,Pay Later,...,0.0,1.0,0.0,0.0,0.0,NaN,2379,466.0,236.0,1448
4,-9221127186162682116,2018-07-23 10:06:00,2018-08-09 00:00:00,2018-08-10 00:00:00,479046,TH,2013-06-06 10:04:00,3.0,Resort,Pay Later,...,0.0,1.0,0.0,0.0,0.0,2018-07-23,1506,NaN,NaN,616


examine the different columns

In [5]:
full_data.T.groupby(full_data.dtypes).apply(lambda df: df.iloc[:,0])

bool     is_user_logged_in                                False
         is_first_booking                                  True
int64    h_booking_id                      -9223194055642672935
         hotel_id                                          6452
         h_customer_id                      6003835896819550002
         guest_is_not_the_customer                            0
         no_of_adults                                         2
         no_of_children                                       0
         no_of_extra_bed                                      0
         no_of_room                                           1
         hotel_area_code                                   1192
         hotel_city_code                                    142
float64  hotel_star_rating                                  4.0
         original_selling_amount                         267.55
         request_nonesmoke                                  NaN
         request_latecheckin            

insights:
    - I don't think that the "original_payment_method" really matters, theres no need to fix "UNKNOWN" values, there is a lot of them
    -

In [14]:
missing_values_count = full_data.isnull().sum()
missing_values_count.loc[missing_values_count > 0]

hotel_country_code           4
origin_country_code          2
request_nonesmoke        25040
request_latecheckin      25040
request_highfloor        25040
request_largebed         25040
request_twinbeds         25040
request_airport          25040
request_earlycheckin     25040
cancellation_datetime    42895
hotel_brand_code         43356
hotel_chain_code         42902
dtype: int64

cleaning some data
    - removing rows with null "hotel country code" or null "origin country code"
    - making the "requests" columns without nulls - replacing that with non-requests, a.k.a zero values
    -
    -

In [49]:
full_data_1 = full_data.loc[(full_data.hotel_country_code.notnull()) & (full_data.origin_country_code.notnull())]
full_data_1[["request_airport", "request_largebed", "request_latecheckin", "request_twinbeds", "request_highfloor", "request_earlycheckin", "request_nonesmoke"]] = full_data_1[["request_airport", "request_largebed", "request_latecheckin", "request_twinbeds", "request_highfloor", "request_earlycheckin", "request_nonesmoke"]].fillna(value=0)



C:\Users\PC\.conda\envs\iml.env\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
0,-9223194055642672935,2018-06-28 21:15:00,2018-07-09 00:00:00,2018-07-13 00:00:00,6452,HK,2009-06-28 02:02:00,4.0,Hotel,Pay Now,...,0.0,0.0,0.0,0.0,0.0,NaN,1192,NaN,NaN,142
1,-9222713784330706132,2018-08-10 22:31:00,2018-08-16 00:00:00,2018-08-17 00:00:00,47729,CN,2011-06-07 11:52:00,4.0,Hotel,Pay Later,...,0.0,0.0,1.0,0.0,0.0,NaN,5617,902.0,600.0,1273
2,-9222411208325704942,2018-09-14 07:55:00,2018-09-14 00:00:00,2018-09-15 00:00:00,780431,KR,2014-11-20 15:43:00,4.0,Hotel,Pay Now,...,0.0,0.0,0.0,0.0,0.0,NaN,3819,NaN,NaN,2799
3,-9222220845872895471,2018-06-25 07:33:00,2018-07-02 00:00:00,2018-07-03 00:00:00,291365,JP,2011-11-21 12:27:00,3.0,Hotel,Pay Later,...,0.0,1.0,0.0,0.0,0.0,NaN,2379,466.0,236.0,1448
4,-9221127186162682116,2018-07-23 10:06:00,2018-08-09 00:00:00,2018-08-10 00:00:00,479046,TH,2013-06-06 10:04:00,3.0,Resort,Pay Later,...,0.0,1.0,0.0,0.0,0.0,2018-07-23,1506,NaN,NaN,616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58654,9221958224902563533,2018-07-18 22:07:00,2018-08-04 00:00:00,2018-08-05 00:00:00,2975682,ID,2017-10-12 11:01:00,2.0,Guest House / Bed & Breakfast,Pay Now,...,0.0,1.0,0.0,0.0,0.0,2018-07-22,995,NaN,NaN,2066
58655,9222015611933681753,2018-09-05 12:07:00,2018-09-06 00:00:00,2018-09-08 00:00:00,3579174,PH,2017-11-30 00:42:00,4.0,Apartment,Pay Now,...,0.0,0.0,0.0,0.0,0.0,NaN,674,NaN,NaN,2310
58656,9222651806661094957,2018-06-08 23:23:00,2018-08-09 00:00:00,2018-08-11 00:00:00,407156,TH,2015-03-14 05:18:00,5.0,Hotel,Pay Now,...,0.0,0.0,0.0,0.0,0.0,NaN,3823,754.0,55.0,2477
58657,9223221736086584899,2018-09-02 08:18:00,2018-09-10 00:00:00,2018-09-12 00:00:00,303186,DE,2012-04-13 17:48:00,0.0,Hostel,Pay Now,...,0.0,0.0,1.0,0.0,0.0,NaN,4663,NaN,NaN,1935


In [46]:
full_data_1

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,request_highfloor,request_largebed,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code
0,-9223194055642672935,2018-06-28 21:15:00,2018-07-09 00:00:00,2018-07-13 00:00:00,6452,HK,2009-06-28 02:02:00,4.0,Hotel,Pay Now,...,NaN,NaN,NaN,NaN,NaN,NaN,1192,NaN,NaN,142
1,-9222713784330706132,2018-08-10 22:31:00,2018-08-16 00:00:00,2018-08-17 00:00:00,47729,CN,2011-06-07 11:52:00,4.0,Hotel,Pay Later,...,0.0,0.0,1.0,0.0,0.0,NaN,5617,902.0,600.0,1273
2,-9222411208325704942,2018-09-14 07:55:00,2018-09-14 00:00:00,2018-09-15 00:00:00,780431,KR,2014-11-20 15:43:00,4.0,Hotel,Pay Now,...,0.0,0.0,0.0,0.0,0.0,NaN,3819,NaN,NaN,2799
3,-9222220845872895471,2018-06-25 07:33:00,2018-07-02 00:00:00,2018-07-03 00:00:00,291365,JP,2011-11-21 12:27:00,3.0,Hotel,Pay Later,...,0.0,1.0,0.0,0.0,0.0,NaN,2379,466.0,236.0,1448
4,-9221127186162682116,2018-07-23 10:06:00,2018-08-09 00:00:00,2018-08-10 00:00:00,479046,TH,2013-06-06 10:04:00,3.0,Resort,Pay Later,...,0.0,1.0,0.0,0.0,0.0,2018-07-23,1506,NaN,NaN,616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58654,9221958224902563533,2018-07-18 22:07:00,2018-08-04 00:00:00,2018-08-05 00:00:00,2975682,ID,2017-10-12 11:01:00,2.0,Guest House / Bed & Breakfast,Pay Now,...,0.0,1.0,0.0,0.0,0.0,2018-07-22,995,NaN,NaN,2066
58655,9222015611933681753,2018-09-05 12:07:00,2018-09-06 00:00:00,2018-09-08 00:00:00,3579174,PH,2017-11-30 00:42:00,4.0,Apartment,Pay Now,...,0.0,0.0,0.0,0.0,0.0,NaN,674,NaN,NaN,2310
58656,9222651806661094957,2018-06-08 23:23:00,2018-08-09 00:00:00,2018-08-11 00:00:00,407156,TH,2015-03-14 05:18:00,5.0,Hotel,Pay Now,...,NaN,NaN,NaN,NaN,NaN,NaN,3823,754.0,55.0,2477
58657,9223221736086584899,2018-09-02 08:18:00,2018-09-10 00:00:00,2018-09-12 00:00:00,303186,DE,2012-04-13 17:48:00,0.0,Hostel,Pay Now,...,0.0,0.0,1.0,0.0,0.0,NaN,4663,NaN,NaN,1935


construct a few new feature columns:
    - sum of special requests -> number
    - same country of "hotel country code" and "customer nationality"
    -

In [39]:
features = np.asarray(["h_booking_id",
                          "hotel_id",
                          "accommadation_type_name",
                          "hotel_star_rating",
                          "customer_nationality"])
labels_colname = "cancellation_datetime"
a = np.append(features,labels_colname)
df = full_data[a]
df.head(8)

,h_booking_id,hotel_id,accommadation_type_name,hotel_star_rating,customer_nationality,cancellation_datetime
0,-9223194055642672935,6452,Hotel,4.0,China,NaN
1,-9222713784330706132,47729,Hotel,4.0,South Africa,NaN
2,-9222411208325704942,780431,Hotel,4.0,South Korea,NaN
3,-9222220845872895471,291365,Hotel,3.0,Singapore,NaN
4,-9221127186162682116,479046,Resort,3.0,Thailand,2018-07-23
5,-9220467519202693449,59272,Resort,3.0,UNKNOWN,2018-05-03
6,-9220037355644368516,400966,Serviced Apartment,3.5,Thailand,NaN
7,-9219746746155641867,69287,Hotel,3.5,Taiwan,2018-07-23


In [52]:
is_canceled = df.cancellation_datetime.notnull()
df.loc['canceled'] = is_canceled.astype(int)
days_after_booking = pd.series()

C:\Users\PC\.conda\envs\iml.env\lib\site-packages\pandas\core\indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


AttributeError: module 'pandas' has no attribute 'series'

In [41]:
df.groupby('hotel_id').apply(lambda df: df.cancellation_datetime)

hotel_id       
1         14669           NaN
          49732           NaN
16        44540    2018-04-29
          57170           NaN
75        36642           NaN
                      ...    
5797970   43586           NaN
5798032   28021           NaN
5799579   22813           NaN
5808200   14892    2018-09-22
5823993   17640    2018-09-26
Name: cancellation_datetime, Length: 58659, dtype: object